https://github.com/Albert-Z-Guo/Deep-Reinforcement-Stock-Trading/tree/master/agents

https://zhuanlan.zhihu.com/p/97856004

在
DQN是解决什么问题？

DQN 解决的是 最优化决策 
问题的变量是哪些？
上一次决策给出的回报，过去的决策给出的回报

优化的操作是：
1. 初始化一个可能的任意固定值
2. 每个时间t，agent 选择一个动作a_t， 得到一个奖励 R_t
3. 进入一个新状态 S_t+1
4. 更新Q值
5. 

输出是：
给出上一次决策，和由上一次决策得到的目前的情况，最优的决策是什么
最终会找到一个最优策略， 即从当前状态开始，所有连续步骤的总回报为回报期望值的最大值是可以实现的。

Q table 告诉我们，在每一种情况下， 每一种行动的价值有多大
适合小规模问题，大规模问题会遇到维度灾难和检索困难。



神经网络中， Q table体现为一个函数，它的功能和Q table 一样，但是这个函数并没有具体地与Q table有映射关系，只是能够给出在同样情况下（state 和 action) 相似的reward 值

% 暂时的理解

输入数据为当前state， 输出数据为对于同一state，所有action的q value（输出为一向量）， 需要将该target Qtable 去匹配 现在输出的Q table

每次输出一个q vector of a situation，一个reward 和一个新的situation。 然后用这个q vector 去匹配 reward 和discounted q value 的和。


![Getting Started](./images/DQN%20algo.png)

expereince replay 的好处
1. 因原输入数据 前后具有连续性， 会导致算法在连续一段时间内朝着同一方向做梯度下降，这样gradient有可能不收敛。 expreince replay 打破该连续性

希望训练数据更加diverse， which leads to different strategy and different results.



In [2]:
import random
from collections import deque

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from utils import Portfolio

In [6]:
import random
from collections import deque

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from utils import Portfolio

class Agent(Portfolio):
    def __init__(self,state_dim,balance,is_eval=False,model_name=""):
        super().__init__(balance=balance)
        self.model_type="DQN"
        self.state_dim = state_dim
        self.action_dim = 3
        self.memory=deque(maxlen=100)
        self.buffer_size=60

        self.gamma=0.95
        self.epsilon=1.0
        self.epsilon_min= 0.01
        self.epsilon_decay = 0.995
        self.is_eval = is_eval
        self.model = load_model("save_models/{}.h5".format(model_name)) if is_eval else self.model()
        self.print_f()
        
        
    def print_f(self):
        print("hello world.")
        return
    
    
    def model(self):
        print("test")
        model=Sequential()
        model.add(Dense(units=64,input_dim = self.state_dim,activation="relu"))
        model.add(Dense(units =32,activation="relu"))
        model.add(Dense(units = 8,activation='relu'))
        model.add(Dense(self.action_dim,activation='softmax'))
        model.compile(loss='mse',optimizer=Adam(lr=0.01))
        return model

    def reset(self):
        self.reset_portfolio()
        self.epsilon=1.0

    def remember(self,state,actions,reward,next_state,done):
        self.memory.append((state,actions,reward,next_state,done))

    def act(self,state):
        if not self.is_eval and np.random.rand() <= self.epsilon:
            return random.randrange(self.action_dim)
        options = self.model.predict(state)
        return np.argmax(options[0])

    def experience_replay(self):

        mini_batch = [self.memory[i] for i in range(len(self.memory)-self.buffer_size+1,len(self.memory))]

        for state, actions, reward, next_state, done in mini_batch:
            if not done:
                Q_target_value = reward+self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                Q_target_value = reward

            next_actions = self.model.predict(state)
            next_actions[0][np.argmax(actions)]=Q_target_value
            history = self.model.fit(state,next_actions,epochs=1,verbose =1)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

        return history.history['loss'][0]











In [7]:
demo  = Agent(13,50000)

test
hello world.


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
demo.print_f()

hello world.
